# CircuS descriptors and ColorAtom
## Enantioselectivity prediction examples

The following tutorial guides through the basic functionality of CircuS descriptors and ColorAtom implemetation. The code can be found on GitHub: https://github.com/POSidorov/ChemInfoTools .

First, the imports of the libraries. Both CircuS descriptors and ColorAtom require CGRtools to work.
For modeling, we use the scikit-learn library,

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from CIMtools.preprocessing import solvent

from CGRtools import RDFRead, MoleculeContainer, ReactionContainer, SDFRead, SMILESRead, smiles, CGRContainer
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedKFold, cross_val_score, KFold, cross_val_predict
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
from sklearn.metrics import mean_absolute_error as mae

from cheminfotools.chem_features import Augmentor, ComplexFragmentor, PassThrough, Pruner

### CircuS descriptors

The data for the exercise is located in the *examples* folder. After reading the data, conversion of SMILES to molecules is necessary. For the full structure of the catalyst (*SMILES* column) and R substitutent a simple transformation is sufficient. However, for Ar substituent we indicate the connection point by a dynamic bond, thus, transformation to CGR is required.

In [4]:
data_THF = pd.read_excel("examples/selectivity_data.xls")

ddg_THF = data_THF["ddG calib (C=0.05)"]

# transforming SMILES to structures - THF
full_structures = []
Ar = []
R = []
for i, row in data_THF.iterrows():
    full_structures.append(smiles(row["SMILES"]))
    full_structures[-1].clean2d()

    r = smiles(row["Ar"])  # The r and p represent the initial and final state for the ggeneration of CGR
    p = smiles(row["Ar"])
    p.delete_bond(1,2)     # to create "product", we remove the indicated bond
    Ar.append(r.compose(p)) # CGR is constructed
    Ar[-1].clean2d()
    R.append(smiles(row["R"]))
    R[-1].clean2d()
    
data_THF["mol_full"] = full_structures
data_THF["mol_Ar"] = Ar
data_THF["mol_R"] = R

To calculate CircuS descriptors, we use the Augmentor class from this library. As an extension of scikit-learn transformer class, it can take alist, an array, or pandas Series containing the molecules and perform the fragmentation, resultsing in a pandas DataFrame of descriptors. The required parameters are the lower and upper limits of the size, format of the input molecules (CGRtools MoleculeContainer or CGRContainer or SMILES). *fit* and *transform* functions are used as usual. 

The descriptors are implemented using CGRtools library and its native substructure extraction functions. Their functionality is the following. The user indicates the desired lower and upper limits for the size of substructures, as the topological radius (number of bonds from a certain atom). Size of 0 means only atom itself, size of 1 – atom and all atoms directly connected to it, and so on. It should be noted that due to the way how substructure extraction is implemented in CGRtools library, the size means the number of atoms from the center, and all the bonds between selected atoms will be present, which may be slightly counterintuitive (see an example for a 5-member ring below). This is repeated for all atoms in the molecule/CGR and for all sizes from lower to upper limit to construct the fragment table.

![Demonstration of CircuS](docs/img/circus-demo1.png)

Let's calculate some descriptors from our structures.

In [5]:
aug = Augmentor(lower=0, upper=4)
aug.fit(data_THF.mol_full)

Augmentor(upper=4)

The SMILES of the fragments can be accessed by the *get_feature_names* function.

In [6]:
aug.get_feature_names()

['F',
 'C',
 'S',
 'O',
 'N',
 'P',
 'CF',
 'FC(F)(F)S',
 'O=S(=O)(C)N',
 'S=O',
 'P=NS',
 'OP(O)(N)=N',
 'PNP',
 'POC',
 'cc(c)O',
 'cc(c)C',
 'ccc',
 'cc(c)c',
 'FC(F)(F)S(=O)(=O)N',
 'FC(F)(F)S(=O)(=O)N=P',
 'OP(O)(N)=NS(=O)(=O)C',
 'COP(OC)(NP)=NS',
 'OP(O)(=N)NP(O)(O)=N',
 'cc(c)OP(O)(N)=N',
 'c(C)(c)c(c(C)c)OP',
 'cc(c)-c(cc)c(c)O',
 'cc(c)cc(C)c',
 'cc(c)c(cc)cc',
 'cc(c)ccc',
 'c(c)ccc',
 'cc(c)c(cc)c(C)c',
 'cc(c)-c(c(c)c)c(c)O',
 'ccc(cc)-c(c)c',
 'c(c)(C)ccc',
 'FC(F)(F)S(=O)(=O)N=P(O)(O)N',
 'FC(F)(F)S(=O)(=O)N=P(OC)(OC)NP',
 'cc1OP(Oc(c)c-c1)(=NS(=O)(=O)C)NP(O)(O)=N',
 'O(C)P(OC)(=NS)NP(OC)(OC)=NS',
 'O1c(c(-ccOP1(=NS)NP)c)c(C)c',
 'cc(c)-c1ccc(c)c2-c(c)cOP(=N)(Oc12)N',
 'ccc(cc)-c1c(c(cc(c)c1)C)OP',
 'cc(c)-c1c(O)cc(c(cc)c1)c',
 'c1c(c2c(cccc2)cc1C)C',
 'c1c(c(ccc1)c)cc',
 'c1cc(ccc1)c',
 'c1(ccccc1c(C)c)c',
 'cc(c)-c1c(ccc2ccccc12)O',
 'cc(c)c1-c2c(cc)c(c)cc(c2OPOc1c)C',
 'c1cc(ccc1)-c(cc)c(O)c',
 'c1ccccc1-c(c)c',
 'c1cc(ccc1)C',
 'c1ccccc1',
 'CC',
 'ccc(cc)C',
 'cc(C)

If a CGR is used to calculate descriptors, the fragments will contain dynamic bonds and/or atoms.

In [7]:
aug.fit(data_THF.mol_Ar)
aug.get_feature_names()

['F',
 'C',
 'S',
 'O',
 'N',
 'P',
 'CF',
 'FC(F)(F)S',
 'O=S(=O)(C)N',
 'S=O',
 'P=NS',
 'OP(O)(N)=N',
 'PNP',
 'POC',
 'cc(c)O',
 'cc(c)C',
 'ccc',
 'cc(c)c',
 'FC(F)(F)S(=O)(=O)N',
 'FC(F)(F)S(=O)(=O)N=P',
 'OP(O)(N)=NS(=O)(=O)C',
 'COP(OC)(NP)=NS',
 'OP(O)(=N)NP(O)(O)=N',
 'cc(c)OP(O)(N)=N',
 'c(C)(c)c(c(C)c)OP',
 'cc(c)-c(cc)c(c)O',
 'cc(c)cc(C)c',
 'cc(c)c(cc)cc',
 'cc(c)ccc',
 'c(c)ccc',
 'cc(c)c(cc)c(C)c',
 'cc(c)-c(c(c)c)c(c)O',
 'ccc(cc)-c(c)c',
 'c(c)(C)ccc',
 'FC(F)(F)S(=O)(=O)N=P(O)(O)N',
 'FC(F)(F)S(=O)(=O)N=P(OC)(OC)NP',
 'cc1OP(Oc(c)c-c1)(=NS(=O)(=O)C)NP(O)(O)=N',
 'O(C)P(OC)(=NS)NP(OC)(OC)=NS',
 'O1c(c(-ccOP1(=NS)NP)c)c(C)c',
 'cc(c)-c1ccc(c)c2-c(c)cOP(=N)(Oc12)N',
 'ccc(cc)-c1c(c(cc(c)c1)C)OP',
 'cc(c)-c1c(O)cc(c(cc)c1)c',
 'c1c(c2c(cccc2)cc1C)C',
 'c1c(c(ccc1)c)cc',
 'c1cc(ccc1)c',
 'c1(ccccc1c(C)c)c',
 'cc(c)-c1c(ccc2ccccc12)O',
 'cc(c)c1-c2c(cc)c(c)cc(c2OPOc1c)C',
 'c1cc(ccc1)-c(cc)c(O)c',
 'c1ccccc1-c(c)c',
 'c1cc(ccc1)C',
 'c1ccccc1',
 'CC',
 'ccc(cc)C',
 'cc(C)

It is possible to limit fragments extracted from CGR to only those that contain dynamic objects, by using the *only_dynamic* option.

In [8]:
aug = Augmentor(lower=0, upper=4, only_dynamic=True)
aug.fit(data_THF.mol_Ar)
aug.get_feature_names()

['cc(c)[->.]C',
 'C[->.]C',
 'ccc(cc)[->.]C',
 'cccc([->.]C)c',
 'c1ccccc1[->.]C',
 'c(c(C)c)c([->.]C)c',
 'c1cc(cc(c1)C)[->.]C',
 'c1cc(ccc1C)[->.]C',
 'cc(c)c([->.]C)cc',
 'cc(c)c(c([->.]C)c)cc',
 'c(c)c1c(cccc1c)[->.]C',
 'c1(c([->.]C)cccc1)c',
 'c1cc(cc(c1)c)[->.]C',
 'c12cccc([->.]C)c1cccc2',
 'c(c)([->.]C)c1ccccc1c',
 'c1cc(ccc1[->.]C)C(C)C',
 'cc(c)cc([->.]C)c',
 'c1cc(ccc1c)[->.]C',
 'c(c1c(cc(cc1)[->.]C)c)c',
 'C[->.]c1cc2ccccc2cc1',
 'c(c1c(ccc(c1)[->.]C)c)c',
 'c1(ccc(cc1c)C)[->.]C',
 'c1(c(ccc(c1)[->.]C)C)c',
 'Cc1c2ccccc2c([->.]C)cc1',
 'CC(C)(C)c1cc([->.]C)ccc1',
 'C1c2ccc(cc2cC1)[->.]C',
 'c1ccc2c3c(CC2)ccc([->.]C)c13',
 'CC(C)(C)c1ccc(cc1)[->.]C',
 'Cc1cc(cc(C)c1)[->.]C',
 'CC(C)c1cc([->.]C)cc(C)c1',
 'CC(C)c1cccc([->.]C)c1',
 'cc(c)c1ccc(cc1c)[->.]C',
 'c1ccc2c(c1c)ccc(c2)[->.]C',
 'c1c(c)-c2ccc(cc2C1)[->.]C',
 'c1c(cc2c(-c(c)c(C2)c)c1)[->.]C',
 'c1cc(cc2c1-ccC2)[->.]C',
 'c1c(ccc2-c(c-cc12)c)[->.]C',
 'c1(c2cccc-3c2c(cc1)-cc-3)[->.]C',
 'cc-1c(c)-c2ccc([->.]C)c3cccc-1

The descriptors are returned as a DataFrame if *transform* function is used.

In [9]:
aug.transform(data_THF.mol_Ar)

,cc(c)[->.]C,C[->.]C,ccc(cc)[->.]C,cccc([->.]C)c,c1ccccc1[->.]C,c(c(C)c)c([->.]C)c,c1cc(cc(c1)C)[->.]C,c1cc(ccc1C)[->.]C,cc(c)c([->.]C)cc,cc(c)c(c([->.]C)c)cc,...,CC(C)(C)c1c(C)cc(cc1)[->.]C,CC1(C)CCC(C)(C)c2ccc(cc12)[->.]C,CC(C)(C)c1cc(ccc1C)[->.]C,C1CCC12c(c(-c3c2cc([->.]C)cc3)c)c,FC(F)(F)c1cc(ccc1)[->.]C,FC(F)(F)c1cc(C)cc([->.]C)c1,C(C)C1(CC)c2cc(ccc2-c(c1c)c)[->.]C,C1CCC2(C1)c(c(-c3ccc([->.]C)cc23)c)c,cc(c)-c1cc(ccc1)[->.]C,ccc(cc)-c1cc(cc([->.]C)c1)C
0,1,1,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,2,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,2,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,2,4,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,2,4,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,2,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,2,4,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,2,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,2,4,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### ComplexFragmentor

The library contains a helper class that concatenates the descriptors calculated by various transformers (for examples, fragments calculated from structures that are located in different columns of a data frame, solvent descriptors, etc). In this case, we will dempnstrate the functionality on an example fo calculating CircuS fragments and solvent descriptors.

ComplexFragmentor class is a scikit-learn compatible transformer that concatenates the features according to specified associations. The most important argument is the *associator* - a dictionary that establishes the correspondence between a column in a data frame X and the transformer that is trained on it.

For example, say you have a data frame with molecules/CGRs in one column ("molecules"), and solvents in another ("solvent"). You want to generate a feture table that includes both structural and solvent descriptors. You would define a ComplexFragmentor class with associator as a dictionary, where keys are column names, and value are the corresponding feature generators. In this case, e.g.,

```
associator = {"molecules": Augmentor(lower=a, upper=b),
            "solvent":SolventVectorizer()}  # see CIMTools library for solvent features
```

ComplexFragmentor assumes that at least one of the types of features will be structural, thus, *structure_columns* parameter defines the columns of the data frame where structures are found.

In this example, let's calculate CircuS descriptors from Ar substituent and concatenate them with the solvent descriptors.

In [10]:
cf = ComplexFragmentor(associator={"mol_Ar":Augmentor(lower=0, upper=4), 
                       # the key in the associator dictionary corresponds to the column name in the data frame
                                  "solvent":solvent.SolventVectorizer()},
                      structure_columns=["mol_Ar"]) # only one column represents a chemical structure

cf.fit(data_THF) # note that we give the full data frame as an argument here

ComplexFragmentor(associator={'mol_Ar': Augmentor(upper=4),
                              'solvent': SolventVectorizer()},
                  structure_columns=['mol_Ar'])

The feature names (SMILES for fragments) are accessed as before.

In [11]:
cf.get_feature_names()

['C',
 'cc(c)[->.]C',
 'C[->.]C',
 'ccc',
 'ccc(cc)[->.]C',
 'cccc([->.]C)c',
 'ccccc',
 'c1ccccc1[->.]C',
 'c1ccccc1',
 'cc(c)C',
 'CC',
 'cccc(C)c',
 'c(c)c(cc)C',
 'c(c(C)c)c([->.]C)c',
 'c1cc(cc(c1)C)[->.]C',
 'c1ccccc1C',
 'c1cc(ccc1C)[->.]C',
 'cc(c)c',
 'cc(c)c([->.]C)cc',
 'cc(c)ccc',
 'cc(c)c(cc)cc',
 'cc(c)c(c([->.]C)c)cc',
 'c(c)c1c(cccc1c)[->.]C',
 'c1(c([->.]C)cccc1)c',
 'c1cc(cc(c1)c)[->.]C',
 'c(c)c1ccccc1c',
 'c12cccc([->.]C)c1cccc2',
 'c1ccccc1c',
 'c(c)([->.]C)c1ccccc1c',
 'CC(C)C',
 'cc(c)C(C)C',
 'CC(C)c(cc)cc',
 'CC(C)c1ccccc1',
 'c1cc(ccc1[->.]C)C(C)C',
 'cc(c)cc([->.]C)c',
 'c1cc(ccc1c)[->.]C',
 'c(c1c(cc(cc1)[->.]C)c)c',
 'c1cccc2ccccc12',
 'C[->.]c1cc2ccccc2cc1',
 'c(c1c(ccc(c1)[->.]C)c)c',
 'cc(c)c(cc)C',
 'cc(c)c(c(C)c)cc',
 'c1(ccc(cc1c)C)[->.]C',
 'c1(c(ccc(c1)[->.]C)C)c',
 'c(c1c(cccc1c)C)c',
 'Cc1c2ccccc2c([->.]C)cc1',
 'cc(c1ccccc1c)C',
 'CC(C)(C)C',
 'CC(C)(C)c(cc)cc',
 'CC(C)(C)c(c)c',
 'CC(C)(C)c1ccccc1',
 'CC(C)(C)c1cc([->.]C)ccc1',
 'CCC',
 'C1c(c(c

Individual transformers can be accessed from the associator.

In [12]:
cf.associator["mol_Ar"].get_feature_names()

['C',
 'cc(c)[->.]C',
 'C[->.]C',
 'ccc',
 'ccc(cc)[->.]C',
 'cccc([->.]C)c',
 'ccccc',
 'c1ccccc1[->.]C',
 'c1ccccc1',
 'cc(c)C',
 'CC',
 'cccc(C)c',
 'c(c)c(cc)C',
 'c(c(C)c)c([->.]C)c',
 'c1cc(cc(c1)C)[->.]C',
 'c1ccccc1C',
 'c1cc(ccc1C)[->.]C',
 'cc(c)c',
 'cc(c)c([->.]C)cc',
 'cc(c)ccc',
 'cc(c)c(cc)cc',
 'cc(c)c(c([->.]C)c)cc',
 'c(c)c1c(cccc1c)[->.]C',
 'c1(c([->.]C)cccc1)c',
 'c1cc(cc(c1)c)[->.]C',
 'c(c)c1ccccc1c',
 'c12cccc([->.]C)c1cccc2',
 'c1ccccc1c',
 'c(c)([->.]C)c1ccccc1c',
 'CC(C)C',
 'cc(c)C(C)C',
 'CC(C)c(cc)cc',
 'CC(C)c1ccccc1',
 'c1cc(ccc1[->.]C)C(C)C',
 'cc(c)cc([->.]C)c',
 'c1cc(ccc1c)[->.]C',
 'c(c1c(cc(cc1)[->.]C)c)c',
 'c1cccc2ccccc12',
 'C[->.]c1cc2ccccc2cc1',
 'c(c1c(ccc(c1)[->.]C)c)c',
 'cc(c)c(cc)C',
 'cc(c)c(c(C)c)cc',
 'c1(ccc(cc1c)C)[->.]C',
 'c1(c(ccc(c1)[->.]C)C)c',
 'c(c1c(cccc1c)C)c',
 'Cc1c2ccccc2c([->.]C)cc1',
 'cc(c1ccccc1c)C',
 'CC(C)(C)C',
 'CC(C)(C)c(cc)cc',
 'CC(C)(C)c(c)c',
 'CC(C)(C)c1ccccc1',
 'CC(C)(C)c1cc([->.]C)ccc1',
 'CCC',
 'C1c(c(c

### Modeling

The modeling with the CircuS fragments follows the same procedure as any other features when using scikit-learn.

In this example, we will use the pre-optimized hyperparameters for a Support Vector Regrssion model. The feature selection will be executed by the Pruner class that takes the results of the genetic algorithm optimizer (see Horvath, et al. Challenges 2014, 5, 450-472. https://doi.org/10.3390/challe5020450). 

In [13]:
def libsvm_parser(params_line):
    params = params_line.split(" ")
    c = float(params[params.index("-c")+1])
    coef0 = float(params[params.index("-r")+1])
    epsilon = float(params[params.index("-p")+1])
    kernel = int(params[params.index("-t")+1])
    if kernel == 0:
        kernel = "linear"
    elif kernel == 1:
        kernel = "poly"
    elif kernel == 2:
        kernel = "rbf"
    elif kernel == 3:
        kernel = "sigmoid"
    
    return SVR(kernel=kernel, gamma="auto", coef0=coef0, C=c, epsilon=epsilon)

In [21]:
cf = ComplexFragmentor(associator={"mol_Ar": Augmentor(lower=0, upper=4),
                                   "solvent": solvent.SolventVectorizer()},
                       structure_columns=["mol_Ar"])

pruner = Pruner("examples/sub-0-4+solvent.pri")

with open("examples/svm.pars") as f:
    model = libsvm_parser(f.read())

pipeline = Pipeline([("fragmentor", cf),
                     ("preprocessing", pruner),
                     ("model", model)])

pipeline.fit(data_THF, ddg_THF)

Pipeline(steps=[('fragmentor',
                 ComplexFragmentor(associator={'mol_Ar': Augmentor(upper=4),
                                               'solvent': SolventVectorizer()},
                                   structure_columns=['mol_Ar'])),
                ('preprocessing',
                 Pruner(prifile='examples/sub-0-4+solvent.pri')),
                ('model',
                 SVR(C=14764.78, coef0=4.3, epsilon=0.320593, gamma='auto'))])

To perform the Leave-One-Out cross-validation, we'll use the calculated features with a blank model.

In [24]:
descs_norm = pipeline[:2].transform(data_THF)

with open("examples/svm.pars") as f:
    loo_model = libsvm_parser(f.read())
    
loo_res = cross_val_predict(loo_model, descs_norm, ddg_THF, cv=len(ddg_THF))

print("MAE =", np.round(mae(ddg_THF,loo_res),3))

MAE = 0.741


### ColorAtom

ColorAtom class implements the approach of calculating atomic contributions to the prediction by a model built using fragment descriptors. In this approach, the weights of all fragments are calculated as partial derivatives of the model’s prediction. To get the weight for one fragment, a new descriptor vector is constructed, where the value of this fragment is different (usually by value of 1 for easier calculation), the property is predicted, and the difference in predictions is taken as the weight. Each atom involved in this fragment accumulates this weight as the score, and the sum of all scores on the atom indicates its importance. This can then be visualized, by assigning colors to positive and negative colors, thus allowing to visually inspect the atomic contributions and draw conclusions which modifications to the structure may be beneficial for further improvement of the studies property.

 The approach is developed and reported in 

> G. Marcou, D. Horvath, V. Solov’ev, A. Arrault, P. Vayer and A. Varnek
> Interpretability of SAR/QSAR models of any complexity by atomic contributions
> Mol. Inf., 2012, 31(9), 639-642, 2012

Current implementation is designed for regression tasks, for models built with Scikit-learn library and using ISIDA fragments implemented in CIMtools or CircuS fragments implemented in chem_features module of this library. 

The application of the ColorAtom requires a trained pipeline containing a fragmentor (both ISIDA and CircuS are supported), features preprocessing and a model. *calculate_atom_contributions* calculates the contributions of each atom for a given molecule and returns them numerically as a dictionary. Otherwise, they can visualized directly in Jupyter Notebook via *output_html* function that returns an HTML table containing an SVG for each structure in the molecule. Since complexFragmentor is also supported, several structures in one data point can be processed simultaneously. 

The coloring is done with matplotlib library. The atom contributions are normalized between 0 and 1 according to the maximum absolute value of the contribution. Therefore, if several structures are present, they will all have their colors normalized by the maximum value amond all contributions. The default colormap is PiYG. The "lower" (more negative) contributions are shown by red color, the "upper" (more positive) - by green. An example can be seen below:

![Demonstration of ColorAtom](docs/img/coloratom-demo1.png)

Let's see how to use the ColorAtom on the previously built model.

In [26]:
from cheminfotools.coloratom import *

ca = ColorAtom(is_complex=True) # since we use ComplexFragmentor, we need to indicate it to the ColorAtom

In [27]:
ca.set_pipeline(pipeline) # by default, ColorAtom will consider that the first position 
                          # in the pipeline is reserved to the fragmentor

For the sake of demonstration, we will use one of the molecules from the initial dataset.

Otherwise, the data needs to be given in the same format as for the pipeline training. So, in our case, it is a data frame with the same columns as before, although only the column that are used in the ComplexFragmentor must be filled.

In [33]:
ca.calculate_atom_contributions(data_THF.iloc[14]) # atom contributions in numerical format

{<CGRtools.containers.cgr.CGRContainer at 0x7f80316c2d60>: {1: -0.916821925312501,
  2: -1.3861373060904114,
  3: -0.5919269517287251,
  4: -0.07550314382982037,
  5: -0.346760835099726,
  6: -0.07550314382982037,
  7: -0.5919269517287251,
  8: -0.5786718922460174,
  9: -0.18327444297150386,
  10: 0.3430533293175557,
  11: 0.579319534715351,
  12: 0.20439899727265498,
  13: 0.579319534715351,
  14: 0.3430533293175557}}

In [34]:
ca.output_html(data_THF.iloc[14]) # atom contributions as a figure (HTML with SVGs)